In [ ]:
# Function to display video in Jupyter Notebook (Kaggle Compatible)
def play_video(file_path, width=800):
    """Display a video file directly in a Jupyter Notebook (Kaggle compatible)."""
    try:
        with open(file_path, "rb") as video_file:
            video_data = video_file.read()
            video_base64 = b64encode(video_data).decode()
    except FileNotFoundError:
        return HTML("<p style='color:red;'>🚨 Error: Video file not found!</p>")
    
    video_html = f"""
    <video width="{width}" controls autoplay loop>
        <source src="data:video/mp4;base64,{video_base64}" type="video/mp4">
        Your browser does not support the video tag.
    </video>
    """
    
    return HTML(video_html)

# Display the compressed video
play_video(compressed_video, width=1000)

## 7. Display Video in Notebook

In [ ]:
# ✅ **Compress the video using FFmpeg**
print("⏳ Compressing video... (This may take a few seconds)")
ffmpeg_command = [
    "ffmpeg", "-i", output_video, "-vcodec", "libx264", "-crf", "28", "-preset", "fast", compressed_video
]
subprocess.run(ffmpeg_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("✅ Compression done!")

## 6. Compress Output Video with FFmpeg

In [ ]:
# Process video frame by frame
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 on the frame
    results = model(frame, verbose=False)  # No extra logs

    # Draw bounding boxes
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0].item()
            label = f"Plate {conf:.2f}"

            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Write frame to output video
    out.write(frame)
    frame_count += 1
    
    if frame_count % 100 == 0:
        print(f"⏳ Processed {frame_count} frames...")

# Release video resources
cap.release()
out.release()
cv2.destroyAllWindows()
print(f"✅ Detection completed! Processed {frame_count} frames.")

## 5. Process Video Frames with YOLOv8

In [ ]:
# Open the input video file
cap = cv2.VideoCapture(input_video)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

print(f"📹 Video Properties - FPS: {fps}, Width: {width}, Height: {height}")

## 4. Initialize Video Capture and Writer

In [ ]:
# Define input and output video paths
input_video = "/kaggle/input/car-number-plate-video/Automatic Number Plate Recognition (ANPR) _ Vehicle Number Plate Recognition (1).mp4"
output_video = "/kaggle/working/output_video.mp4"
compressed_video = "/kaggle/working/output_video_compressed.mp4"  # Compressed version

## 3. Configure Video Paths

In [ ]:
model_path = "/kaggle/working/runs/detect/train/weights/best.pt"
model = YOLO(model_path)

## 2. Load YOLOv8 Model

In [ ]:
import cv2
import subprocess
from ultralytics import YOLO
from IPython.display import HTML
from base64 import b64encode

## 1. Import Required Libraries

# ANPR Video Detection with YOLOv8

This notebook processes video frames with YOLOv8 object detection, identifies license plates, and displays the compressed results.